In [11]:
%matplotlib qt
from bluesky import RunEngine
from bluesky.callbacks import LiveGrid
from bluesky.plans import grid_scan
from bluesky.plan_stubs import kickoff, complete, collect, \
    open_run, close_run, mv, one_nd_step, trigger_and_read, move_per_step
from ophyd import Device
from ophyd.flyers import FlyerInterface
from ophyd.status import SubscriptionStatus
from ophyd.sim import motor1, motor2, det4
from matplotlib.pyplot import ion
from bluesky.utils import install_nb_kicker
install_nb_kicker()

ion()
RE = RunEngine()

## Fly Scan

In [2]:
class FlyerDevice(FlyerInterface, Device):
    
    def kickoff(self):
        print("Start asynchronous task")
        return motor1.set(3)

    def fly_monitor(self, **kwargs):
        position = kwargs["obj"].readback.get()
        print(f"Monitoring: {position}")
        if position == 3:
            return True
        return False

    def complete(self):
        return SubscriptionStatus(motor1, callback=self.fly_monitor)
    
    def describe_collect(self):
        return {"decription": {}}
    
    def collect(self):
        print("Collecting data after scan")
        return [{"data": {}, "timestamps": {}, "time": 0, "seq_num": 0}]

In [3]:
def flyscan(flyer):
    yield from open_run()
    yield from kickoff(flyer, wait=True)
    yield from complete(flyer, wait=True)
    yield from collect(flyer)
    yield from close_run()
    
flyer_device = FlyerDevice(name="flyer_device")
RE(mv(motor1, 5))
RE(flyscan(flyer_device))

Start asynchronous task
Monitoring: 3.0


('bf336d4c-facd-452e-99b4-674a123a8afe',)

## per_step

In [24]:
def custom_step(detectors, step, pos_cache):
    print("--------------------------")
    print(f"Grid step: {step.values()}")
    print("Before plan step")
    yield from one_nd_step(detectors, step, pos_cache)
    print("After plan step")

In [25]:
RE(
    grid_scan([det4], motor1, -1, 1, 5, motor2, -2, 2, 5, per_step=custom_step),
    LiveGrid((5, 5), 'det4'))

--------------------------
Grid step: dict_values([-1.0, -2.0])
Before plan step
After plan step
--------------------------
Grid step: dict_values([-1.0, -1.0])
Before plan step
After plan step
--------------------------
Grid step: dict_values([-1.0, 0.0])
Before plan step
After plan step
--------------------------
Grid step: dict_values([-1.0, 1.0])
Before plan step
After plan step
--------------------------
Grid step: dict_values([-1.0, 2.0])
Before plan step
After plan step
--------------------------
Grid step: dict_values([-0.5, -2.0])
Before plan step
After plan step
--------------------------
Grid step: dict_values([-0.5, -1.0])
Before plan step
After plan step
--------------------------
Grid step: dict_values([-0.5, 0.0])
Before plan step
After plan step
--------------------------
Grid step: dict_values([-0.5, 1.0])
Before plan step
After plan step
--------------------------
Grid step: dict_values([-0.5, 2.0])
Before plan step
After plan step
--------------------------
Grid step

('a05ca109-8352-47a5-8e45-98321ee36054',)

- trigger_and_read: Triggera e lê uma lista de detectores e motores e armazena seus valores.
- move_per_step: Envia setpoints para vários dispositivos que permitem ser setados e espera eles chegaram na posição.
- one_shot: Step para o plano 'count'
- one_1d_step: Step para planos com 1 Dimensão
- one_nd_step: Step para planos com N Dimensões

In [ ]:
def one_shot(detectors, motor, step):
    yield from trigger_and_read(list(detectors) + [motor])

In [ ]:
def one_1d_step(detectors, motor, step):
    yield from move()
    yield from trigger_and_read(list(detectors) + [motor])

In [ ]:
def one_nd_step(detectors, step, pos_cache):
    yield from move_per_step(step, pos_cache)
    yield from trigger_and_read(list(detectors) + list(motors))